# Gravix Layer Cookbook: Agentic Email Triage Workflow
This notebook provides a step-by-step guide to building a complete **Agentic Email Triage** workflow using the **Gravix Layer** API, IMAPClient, and intelligent email processing. You will learn how to create an application that automatically fetches, analyzes, prioritizes, and summarizes emails using agentic orchestration and LLM-powered content analysis.

### What is Agentic Email Triage?
Agentic Email Triage uses a modular, agent-based approach to automate email management. Instead of manually sorting through emails, the workflow orchestrates multiple specialized agents to fetch emails, extract meaningful content, analyze importance, and generate actionable summaries with priority classifications.

### In this notebook, you will learn to:
1. **Set up** the environment and configure Gmail IMAP access with SSL security.
2. **Fetch** and process emails using IMAPClient with robust MIME parsing.
3. **Extract** clean text content from complex email formats (HTML, multipart, etc.).
4. **Analyze** email content using Gravix Layer LLM for intelligent summarization.
5. **Prioritize** emails automatically based on content, urgency, and action requirements.
6. **Generate** structured reports with DataFrames and Markdown digests.

### Key Features:
- **Robust Email Parsing**: Handles complex MIME structures, HTML cleanup, and character encoding
- **Intelligent Content Analysis**: LLM-powered summarization that understands email context
- **Priority Classification**: Automatic categorization into High/Medium/Low priority levels
- **Structured Output**: Both tabular (DataFrame) and readable (Markdown) formats
- **Error Handling**: Graceful fallbacks for parsing failures and connection issues

### Use Cases:
- Personal email management and daily digest generation
- Business email monitoring and priority filtering
- Customer service email triage and routing
- Newsletter and notification summarization
- Email analytics and trend analysis

### Disclaimer
**This notebook is for educational and automation purposes only. Always review automated email summaries before taking action. The system processes email content using AI models that may have limitations in understanding context or sensitive information. Ensure compliance with your organization's email policies and privacy requirements when implementing automated email processing.**

In [51]:
# Install required packages
!pip install imapclient certifi rich pandas gravixlayer


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## 📧 Setup and Configuration

First, let's import the required libraries and set up our email credentials and API connections.

### Required Dependencies:
- **IMAPClient**: Secure Gmail IMAP connection with SSL
- **GravixLayer**: LLM client for intelligent email analysis
- **Rich**: Beautiful console output and Markdown rendering
- **Pandas**: Structured data handling for email reports
- **Email Libraries**: Robust MIME parsing and content extraction

### Security Notes:
- Use Gmail App Passwords (not your main password)
- Store credentials in environment variables for production
- SSL/TLS encryption for all email connections

In [ ]:
import os
import re
import json
import ssl
import certifi
from datetime import datetime, timedelta
from imapclient import IMAPClient
from rich.console import Console
from rich.markdown import Markdown
import pandas as pd
from gravixlayer import GravixLayer
from email import policy
from email import message_from_bytes
from email.header import decode_header, make_header

console = Console()


EMAIL = "enter your email id here" 
APP_PASSWORD = "enter your app password here"

if not EMAIL or not APP_PASSWORD:
    console.print("[bold yellow]⚠️  Warning:[/bold yellow] Set GMAIL_ADDRESS and GMAIL_APP_PASSWORD before running fetch.")

# Initialize GravixLayer client
try:
    gravix_client = GravixLayer() #ENSURE TO GET YOUR API KEY FROM https://platform.gravixlayer.com and set it in your environment variable GRAVIXLAYER_API_KEY
    console.print("[bold green]✅ GravixLayer client initialized successfully[/bold green]")
except Exception as e:
    console.print(f"[bold red]❌ GravixLayer initialization failed:[/bold red] {e}")
    console.print("[bold yellow]💡 Tip:[/bold yellow] Make sure GRAVIXLAYER_API_KEY is set in your environment")

✅ GravixLayer client initialized successfully

## 🛠️ Email Processing Utilities

These utility functions handle the complex task of extracting clean, readable text from various email formats. Email content can be challenging to parse due to:
- **MIME multipart structures** (text, HTML, attachments)
- **Character encoding issues** (UTF-8, ISO-8859-1, etc.)
- **HTML content** with embedded styles and scripts
- **Email headers** with encoded subjects
- **Boilerplate content** (unsubscribe links, signatures)

### Key Functions:
1. **`extract_plain_text_from_email()`**: Robust MIME parsing with HTML fallback
2. **`decode_subject_value()`**: Handles encoded email subjects 
3. **`build_digest_markdown()`**: Generates readable priority-grouped reports
4. **JSON extraction helpers**: Parse LLM responses reliably

In [53]:
def _ensure_text(val):
    """Convert various input types to clean text strings."""
    if isinstance(val, list):
        return " ".join(str(v) for v in val)
    if val is None:
        return ""
    return str(val)

def decode_subject_value(subject_raw: str) -> str:
    """
    Decode email subjects that may contain encoded characters.
    
    Examples:
    - "=?UTF-8?B?SGVsbG8gV29ybGQ=?=" → "Hello World"
    - "=?ISO-8859-1?Q?Caf=E9?=" → "Café"
    """
    try:
        return str(make_header(decode_header(subject_raw or "")))
    except Exception:
        return subject_raw or ""

def extract_plain_text_from_email(raw_message_bytes: bytes) -> str:
    """
    Robustly extract readable text from a full RFC822 message.
    
    Processing Strategy:
    1. Parse MIME multipart structure
    2. Prefer text/plain parts (skip attachments)
    3. Fallback to text/html with tag stripping
    4. Clean boilerplate content (unsubscribe links, etc.)
    5. Normalize whitespace and formatting
    
    Args:
        raw_message_bytes: Complete RFC822 email message
        
    Returns:
        Clean, readable text content
    """
    if not raw_message_bytes:
        return "No content found."

    msg = message_from_bytes(raw_message_bytes, policy=policy.default)

    def decode_part(p):
        """Safely decode email part content with charset handling."""
        try:
            return p.get_content()
        except Exception:
            payload = p.get_payload(decode=True) or b""
            charset = p.get_content_charset() or "utf-8"
            return payload.decode(charset, errors="replace")

    text_parts = []
    html_fallback = None

    # Handle multipart messages (most common)
    if msg.is_multipart():
        for part in msg.walk():
            # Skip attachments
            if part.get_content_disposition() == "attachment":
                continue
            ctype = part.get_content_type()
            if ctype == "text/plain":
                text_parts.append(decode_part(part))
            elif ctype == "text/html" and html_fallback is None:
                html_fallback = decode_part(part)
    else:
        # Handle single-part messages
        ctype = msg.get_content_type()
        if ctype == "text/plain":
            text_parts.append(decode_part(msg))
        elif ctype == "text/html":
            html_fallback = decode_part(msg)

    text = "\n\n".join([t for t in text_parts if t]).strip()

    # Fallback to HTML conversion if no plain text
    if not text and html_fallback:
        html = html_fallback
        # Remove script and style content
        html = re.sub(r"(?is)<(script|style).*?>.*?</\1>", " ", html)
        # Convert HTML breaks and paragraphs
        html = re.sub(r"(?is)<br\s*/?>", "\n", html)
        html = re.sub(r"(?is)</p\s*>", "\n\n", html)
        # Strip all remaining HTML tags
        html = re.sub(r"(?is)<.*?>", " ", html)
        # Decode HTML entities
        html = html.replace("&nbsp;", " ").replace("&amp;", "&").replace("&lt;", "<").replace("&gt;", ">")
        text = html

    # Clean boilerplate and formatting
    lines = []
    for line in (text or "").splitlines():
        s = line.strip()
        if not s:
            lines.append("")
            continue
        # Remove common email boilerplate
        if re.search(r"unsubscribe|view in browser|do not reply|privacy policy", s, re.I):
            continue
        # Remove divider lines
        if re.match(r"^-{2,}$", s):
            continue
        lines.append(s)

    cleaned = "\n".join(lines)
    # Normalize spacing
    cleaned = re.sub(r"[ \t]+", " ", cleaned)
    cleaned = re.sub(r"\n{3,}", "\n\n", cleaned)
    cleaned = cleaned.strip()
    
    return cleaned or "No content found."

def build_digest_markdown(date_label, results):
    """
    Generate a structured Markdown digest grouped by priority.
    
    Args:
        date_label: Date string for the digest header
        results: List of processed email summaries
        
    Returns:
        Formatted Markdown string with priority sections
    """
    md = f"# 📧 Email Digest for {date_label}\n\n"
    
    # Group by priority level
    grouped = {"High": [], "Medium": [], "Low": []}
    for r in results:
        grouped.setdefault(r.get("priority", "Low"), []).append(r)
    
    # Generate sections by priority
    priority_icons = {"High": "🔴", "Medium": "🟡", "Low": "🟢"}
    for level in ["High", "Medium", "Low"]:
        if grouped[level]:
            md += f"## {priority_icons[level]} {level} Priority ({len(grouped[level])} emails)\n\n"
            for r in grouped[level]:
                line = f"### 📨 From: {r.get('from','')}  |  Subject: {r.get('subject','')}\n"
                summary = _ensure_text(r.get("summary"))
                actions = _ensure_text(r.get("actions"))
                if summary:
                    line += f"**Summary:** {summary.strip()}\n\n"
                if actions:
                    line += f"**Action Items:** {actions.strip()}\n\n"
                if r.get("duplicate_of"):
                    line += f"*⚠️ Possible duplicate of: {r['duplicate_of']}*\n\n"
                md += line + "---\n\n"
    
    return md

def _strip_code_fences(text: str) -> str:
    """Remove markdown code fence markers from text."""
    t = text.strip()
    t = re.sub(r"^\s*```.*?\n", "", t)
    t = re.sub(r"\s*```\s*$", "", t)
    return t

def _extract_first_json_object(text: str) -> str:
    """
    Robustly extract the first complete JSON object from text.
    
    Handles LLM responses that may include explanatory text
    before or after the JSON object.
    """
    s = _strip_code_fences(text)
    start = None
    depth = 0
    for i, ch in enumerate(s):
        if ch == '{':
            if depth == 0:
                start = i
            depth += 1
        elif ch == '}' and depth > 0:
            depth -= 1
            if depth == 0 and start is not None:
                return s[start:i+1]
    raise ValueError("No JSON object found")

## Agentic Email Processing Architecture

The email triage system uses a **modular agent architecture** where each agent has a specific responsibility:

### Agent Workflow:
```
EmailFetcherAgent → SummarizerAgent → ReporterAgent
     │                       │                    │
     └─ Secure IMAP          └─ LLM Analysis      └─ Structured Output
        Connection              & Prioritization     (DataFrame + Markdown)
```

### Agent Responsibilities:

#### 1. **EmailFetcherAgent**
- **Purpose**: Secure email retrieval and initial processing
- **Features**: SSL/TLS connections, MIME parsing, date filtering
- **Output**: Structured email objects with clean text content

#### 2. **SummarizerAgent**  
- **Purpose**: Intelligent content analysis and prioritization
- **Features**: LLM-powered summarization, priority classification, action extraction
- **Model**: Uses Gravix Layer with Llama 3.1 8B for accurate understanding

#### 3. **ReporterAgent**
- **Purpose**: Generate structured reports and visualizations
- **Features**: Priority grouping, DataFrame creation, Markdown formatting
- **Output**: Both machine-readable (CSV/DataFrame) and human-readable (Markdown) formats

### Configuration Options:
- **Date Range**: Configurable lookback period for email fetching
- **Email Limits**: Batch processing with configurable limits
- **Priority Levels**: High/Medium/Low with customizable criteria
- **Output Formats**: Multiple export options for different use cases

In [ ]:
class EmailFetcherAgent:
    """
    Secure Email Fetching Agent
    
    Handles Gmail IMAP connections with SSL security and robust error handling.
    Fetches emails from specified date ranges and processes MIME content.
    """
    
    def __init__(self, email_address: str, app_password: str):
        """
        Initialize the email fetcher with credentials.
        
        Args:
            email_address: Gmail address for IMAP access
            app_password: Gmail App Password (not regular password)
        """
        self.email = email_address
        self.app_password = app_password

    def fetch_unread_emails(self, days_back: int = 1, limit: int = 10) -> list:
        """
        Fetch emails from Gmail using IMAP with SSL security.
        
        Args:
            days_back: Number of days to look back for emails
            limit: Maximum number of emails to fetch
            
        Returns:
            List of processed email dictionaries with subject, from, and body
        """
        target_date = (datetime.now() - timedelta(days=days_back)).strftime('%d-%b-%Y')
        
        # Create secure SSL context using certifi certificates
        context = ssl.create_default_context(cafile=certifi.where())
        
        try:
            with IMAPClient("imap.gmail.com", ssl=True, ssl_context=context) as client:
                # Authenticate with Gmail
                client.login(self.email, self.app_password)
                client.select_folder("INBOX")
                
                # Search for emails from target date
                messages = client.search(["ON", target_date])[:limit]
                
                if not messages:
                    console.print(f"[yellow]No emails found for {target_date}[/yellow]")
                    return []
                
                # Fetch email content and metadata
                response = client.fetch(messages, ["ENVELOPE", "RFC822"])
                emails = []
                
                for msgid, data in response.items():
                    try:
                        # Extract envelope data (headers)
                        envelope = data[b"ENVELOPE"]
                        subject_raw = envelope.subject.decode(errors="ignore") if envelope.subject else "(No Subject)"
                        subject = decode_subject_value(subject_raw)
                        
                        # Extract sender information
                        if envelope.from_ and len(envelope.from_) > 0:
                            from_addr = envelope.from_[0]
                            from_ = f"{from_addr.mailbox.decode()}@{from_addr.host.decode()}"
                        else:
                            from_ = "unknown@unknown.com"
                        
                        # Process email body
                        raw_bytes = data[b"RFC822"]
                        clean_body = extract_plain_text_from_email(raw_bytes)
                        
                        emails.append({
                            "subject": subject,
                            "from": from_,
                            "body": clean_body,
                            "message_id": msgid
                        })
                        
                    except Exception as e:
                        console.print(f"[red]Error processing email {msgid}: {e}[/red]")
                        continue
                
                console.print(f"[green]✅ Fetched {len(emails)} emails from {target_date}[/green]")
                return emails
                
        except Exception as e:
            console.print(f"[bold red]❌ IMAP connection failed:[/bold red] {e}")
            return []


class SummarizerAgent:
    """
    Intelligent Email Summarization Agent
    
    Uses Gravix Layer LLM to analyze email content and generate:
    - Meaningful summaries based on actual content
    - Priority classifications (High/Medium/Low)
    - Action items and next steps
    """
    
    def __init__(self, llm_client):
        """
        Initialize with LLM client for content analysis.
        
        Args:
            llm_client: GravixLayer client instance
        """
        self.client = llm_client

    def _normalize_record(self, rec: dict, src: dict) -> dict:
        """
        Normalize and validate LLM response data.
        
        Args:
            rec: Raw LLM response dictionary
            src: Original email data for fallback
            
        Returns:
            Clean, validated email record
        """
        def _t(x):
            return "" if x is None else (x if isinstance(x, str) else str(x))
        
        # Handle encoded subjects
        subj = _t(rec.get("subject", src["subject"]))
        if re.search(r"=\?.+\?=", subj):
            subj = decode_subject_value(subj)
        
        # Build normalized record
        out = {
            "from": _t(rec.get("from", src["from"])),
            "subject": subj,
            "priority": _t(rec.get("priority", "Low")).title(),
            "summary": _t(rec.get("summary", "")),
            "actions": _t(rec.get("actions", ""))
        }
        
        # Validate priority level
        if out["priority"] not in {"High", "Medium", "Low"}:
            out["priority"] = "Low"
        
        # Handle duplicate detection
        dup = rec.get("duplicate_of")
        if dup is not None:
            out["duplicate_of"] = _t(dup)
        
        return out

    def summarize_and_prioritize(self, emails: list) -> list:
        """
        Analyze emails using LLM for intelligent summarization and prioritization.
        
        Args:
            emails: List of email dictionaries to process
            
        Returns:
            List of processed emails with summaries and priorities
        """
        if not emails:
            console.print("[yellow]No emails to summarize[/yellow]")
            return []
        
        results = []
        console.print(f"[blue] Analyzing {len(emails)} emails with LLM...[/blue]")
        
        for i, e in enumerate(emails, 1):
            console.print(f"[dim]Processing email {i}/{len(emails)}...[/dim]")
            
            # Construct detailed analysis prompt
            prompt = f"""You are an expert email assistant. Analyze the following email and provide a JSON response.

IMPORTANT: Read the actual email content carefully and create a meaningful summary based on what the email is actually about.

Email to analyze:
From: {e['from']}
Subject: {e['subject']}
Body: {e['body']}

Return ONLY this JSON format (no markdown, no explanation):
{{
  "from": "{e['from']}",
  "subject": "{e['subject']}",
  "priority": "High|Medium|Low",
  "summary": "2-3 sentences describing what this email is actually about based on the content above",
  "actions": "specific actions mentioned or implied in the email, if any"
}}

Priority Guidelines:
- **High**: Urgent, time-sensitive, requires immediate action, deadlines, emergencies
- **Medium**: Important but not urgent, informational with some action needed, meetings, updates
- **Low**: FYI, newsletters, notifications that don't require action, social media, marketing

Focus on the actual content and purpose of the email, not generic descriptions."""

            try:
                # Call LLM for analysis
                resp = self.client.chat.completions.create(
                    model="meta-llama/llama-3.1-8b-instruct",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.1,  # Low temperature for consistent analysis
                    max_tokens=300
                )
                
                raw = resp.choices[0].message.content or ""
                
                # Parse JSON response
                try:
                    js = _extract_first_json_object(raw)
                    parsed = json.loads(js)
                except Exception:
                    # Fallback JSON parsing
                    parsed = json.loads(_strip_code_fences(raw))
                
                # Normalize and validate response
                normalized = self._normalize_record(parsed, e)
                results.append(normalized)
                
            except Exception as parse_error:
                console.print(f"[red]⚠️  LLM parsing failed for email {i}: {parse_error}[/red]")
                # Fallback to basic processing
                results.append({
                    "from": e["from"],
                    "subject": e["subject"],
                    "priority": "Low",
                    "summary": (e["body"] or "")[:300] + "..." if len(e["body"] or "") > 300 else (e["body"] or ""),
                    "actions": ""
                })
        
        console.print(f"[green]✅ Completed analysis of {len(results)} emails[/green]")
        return results


class ReporterAgent:
    """
    📊 Email Report Generation Agent
    
    Creates structured reports in multiple formats:
    - DataFrame for programmatic access
    - Markdown for human-readable digests
    - Priority-based organization
    """
    
    def __init__(self, date_label: str):
        """
        Initialize reporter with date context.
        
        Args:
            date_label: Date string for report headers
        """
        self.date_label = date_label

    def run(self, summaries: list) -> tuple:
        """
        Generate comprehensive email reports.
        
        Args:
            summaries: List of analyzed email summaries
            
        Returns:
            Tuple of (DataFrame, Markdown string)
        """
        if not summaries:
            empty_df = pd.DataFrame(columns=["from", "subject", "summary", "actions", "priority"])
            empty_md = f"# 📧 Email Digest for {self.date_label}\n\n*No emails to display.*"
            return empty_df, empty_md
        
        # Prepare structured data
        out = []
        for r in summaries:
            out.append({
                "from": r.get("from", ""),
                "subject": r.get("subject", ""),
                "summary": _ensure_text(r.get("summary")),
                "actions": _ensure_text(r.get("actions")),
                "priority": r.get("priority", "Low"),
                "duplicate_of": r.get("duplicate_of", None)
            })
        
        # Sort by priority (High → Medium → Low), then by sender
        priority_order = {"High": 0, "Medium": 1, "Low": 2}
        out.sort(key=lambda r: (
            priority_order.get(r["priority"], 3),
            r["from"].lower(),
            r["subject"].lower()
        ))
        
        # Create DataFrame
        df = pd.DataFrame(out)
        
        # Generate Markdown digest
        md = build_digest_markdown(self.date_label, out)
        
        # Add summary statistics
        stats = df['priority'].value_counts().to_dict()
        console.print(f"[blue]📈 Email Summary: {stats.get('High', 0)} High, {stats.get('Medium', 0)} Medium, {stats.get('Low', 0)} Low priority[/blue]")
        
        return df, md

## Complete Email Triage Workflow

Now let's run the complete agentic email triage workflow. This will:

1. **Authenticate** with Gmail using secure IMAP/SSL
2. **Fetch** emails from the specified date range  
3. **Analyze** content using Gravix Layer LLM
4. **Generate** structured reports and digestsand 


### Expected Output:
- **Rich Console Display**: Color-coded priority digest with emojis
- **DataFrame Table**: Structured data for further analysis
- **Markdown Report**: Human-readable priority-grouped summary

In [ ]:
# Execute Complete Email Triage Workflow

console.rule("[bold blue]🤖 Starting Agentic Email Triage Workflow[/bold blue]")

# Step 1: 📧 Initialize and fetch emails
console.print("[bold blue]Step 1: Fetching emails...[/bold blue]")
fetcher = EmailFetcherAgent(EMAIL, APP_PASSWORD)
emails = []

try:
    if EMAIL and APP_PASSWORD:
        emails = fetcher.fetch_unread_emails(days_back=1, limit=10)
    else:
        console.print("[bold yellow]⚠️  Skipping fetch: credentials not set[/bold yellow]")
        console.print("[dim]Set EMAIL and APP_PASSWORD variables to enable email fetching[/dim]")
except Exception as e:
    console.print(f"[bold red]❌ Fetch error:[/bold red] {e}")

# Step 2: Analyze emails with LLM
console.print(f"\n[bold blue]Step 2: Analyzing {len(emails)} emails with LLM...[/bold blue]")
summarizer = SummarizerAgent(gravix_client)
summaries = summarizer.summarize_and_prioritize(emails)

# Step 3: Generate reports
console.print(f"\n[bold blue]Step 3: Generating reports...[/bold blue]")
date_label = (datetime.now() - timedelta(days=1)).strftime('%d-%b-%Y')
reporter = ReporterAgent(date_label=date_label)
df, md = reporter.run(summaries)

# Step 4: Display results
console.print(f"\n[bold green]✅ Workflow Complete![/bold green]")
console.rule("[bold blue]📧 Prioritized Email Digest[/bold blue]")
console.print(Markdown(md))

# Display DataFrame preview
if not df.empty:
    console.print(f"\n[bold blue]📈 DataFrame Preview ({len(df)} emails):[/bold blue]")
    console.print("[dim]Full data available in 'df' variable for further analysis[/dim]")

# Return DataFrame for inspection
df.head(10) if not df.empty else console.print("[yellow]No emails to display[/yellow]")

──────────────────────────────────── 🤖 Starting Agentic Email Triage Workflow ────────────────────────────────────

Step 1: Fetching emails...

✅ Fetched 10 emails from 06-Sep-2025

Step 2: Analyzing 10 emails with LLM...

🧠 Analyzing 10 emails with LLM...

...

─────────────────────────────────────────── 📧 Prioritized Email Digest ───────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                         📧 Email Digest for 06-Sep-2025                                         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                           🟡 Medium Priority (9 emails)                                           

📨 From: express@jobagent.stepstone.de  |  Subject: You have the skills for this job: Junior Managers Program - HR 
                                          - with a focus on HR tech, d...                                          

Summary: The email is about a job opportunity at StepStone, specifically the Junior Managers Program in HR with a  
focus on HR tech.                                                                                                  

Action Items: Apply for the job or visit the company's social media profiles                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 📨 From: jobalerts-noreply@linkedin.com  |  Subject: “software engineer”: AfterQuery Experts - Software Engineer  
                                                 posted on 9/5/25                                                  

Summary: This email contains job alerts for software engineering positions in Hyderabad, including a new job       
posting at AfterQuery Experts.                                                                                     

Action Items: ['View the job postings: https://www.linkedin.com/comm/jobs/view/[job ID]', 'Apply with resume and   
profile']                                                                                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   📨 From: newsletter@opencv.org  |  Subject: Here's Your OpenCV Friday Update!                   

Summary: OpenCV has lowered prices for merchandise, a new newsletter is available with updates and projects to     
explore.                                                                                                           

Action Items: Shop Official OpenCV Merchandise, Explore Cool OpenCV Projects on Hackster                           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        📨 From: no-reply@outlier.ai  |  Subject: Inside Outlier: Finance expertise and your Outlier update        

Summary: The email promotes a platform called Outlier that allows finance experts to monetize their knowledge by   
training AI systems, with flexible scheduling and competitive rates.                                               

Action Items: ["Apply now to start training tomorrow's financial AI", 'Email insideoutlier@outlier.ai for the      
chance to be featured in the next newsletter', 'Send a photo of your window view alongside your Outlier dashboard']

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
              📨 From: no-reply@zoomin.com  |  Subject: Personalized Ganesha Frames & Décor from ₹239              

Summary: Zoomin is offering a 20% discount on personalized Ganesha frames and decor, with various products such as 
photo magnets, calendars, posters, and acrylic prints available for purchase.                                      

Action Items: Use code BDAYSALE to get 20% off sitewide, shop now for specific products                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        📨 From: noreply@dare2compete.news  |  Subject: [Congrats] Nokia is hiring - Earn a CTC of 15 LPA!         

Summary: This email promotes job openings at 

📈 DataFrame Preview (10 emails):

Full data available in 'df' variable for further analysis

,from,subject,summary,actions,priority,duplicate_of
0,express@jobagent.stepstone.de,You have the skills for this job: Junior Manag...,The email is about a job opportunity at StepSt...,Apply for the job or visit the company's socia...,Medium,None
1,jobalerts-noreply@linkedin.com,“software engineer”: AfterQuery Experts - Soft...,This email contains job alerts for software en...,['View the job postings: https://www.linkedin....,Medium,None
2,newsletter@opencv.org,Here's Your OpenCV Friday Update!,"OpenCV has lowered prices for merchandise, a n...","Shop Official OpenCV Merchandise, Explore Cool...",Medium,None
3,no-reply@outlier.ai,Inside Outlier: Finance expertise and your Out...,The email promotes a platform called Outlier t...,"[""Apply now to start training tomorrow's finan...",Medium,None
4,no-reply@zoomin.com,Personalized Ganesha Frames & Décor from ₹239,Zoomin is offering a 20% discount on personali...,"Use code BDAYSALE to get 20% off sitewide, sho...",Medium,None
5,noreply@dare2compete.news,[Congrats] Nokia is hiring - Earn a CTC of 15 ...,This email promotes job openings at various co...,"Apply for jobs, complete profile on Unstop",Medium,None
6,noreply@emails.unstop.com,Hiring Alert! | Python Development Internship ...,InternshipHub is seeking a highly motivated an...,Apply Now,Medium,None
7,reminders@123greetings.com,Anniversary of rupin.manthan@gmail.com is tomo...,This email is a reminder to send an anniversar...,"Send an anniversary ecard, Download the 123Gre...",Medium,None
8,updates@simplilearnmailer.com,From Python to GenAI - Become Job-Ready with P...,The email promotes an online course on generat...,Enroll now in the Applied Generative AI Specia...,Medium,None
9,tatacliq@mall.tatacliq.com,"Crafted by India, Loved by You",This email is a promotional message from Tata ...,Download The App,Low,None


## 🧪 Advanced Features & Customization

### 🔧 Testing Different Date Ranges
You can easily test the system with different date ranges to find emails for processing:

```python
# Test with different date ranges
fetcher.fetch_unread_emails(days_back=7, limit=5)   # Last week
fetcher.fetch_unread_emails(days_back=30, limit=15) # Last month
```

### 🎯 Customizing Priority Logic
Modify the SummarizerAgent prompt to adjust priority classification:

```python
# Add custom priority criteria in the prompt:
# - High: Customer complaints, security alerts, deadline reminders
# - Medium: Team updates, meeting requests, project status
# - Low: Newsletters, social notifications, marketing emails
```

### 📊 Export Options
The system generates multiple output formats for different use cases:

- **DataFrame (`df`)**: Perfect for data analysis, CSV export, or integration with other systems
- **Markdown (`md`)**: Human-readable reports for documentation or sharing
- **Rich Console**: Interactive terminal display with colors and formatting

### 🔍 Analysis Examples
Once you have the DataFrame, you can perform advanced analysis:

```python
# Priority distribution
priority_counts = df['priority'].value_counts()

# Most active senders
top_senders = df['from'].value_counts().head(10)

# Export to CSV
df.to_csv('email_digest.csv', index=False)

# Filter high priority emails
urgent_emails = df[df['priority'] == 'High']
```

In [50]:
# 🧪 Advanced Testing: Custom Date Range Analysis

console.rule("[bold cyan]🧪 Advanced Testing: Custom Date Range[/bold cyan]")

def test_custom_date_range(days_back: int = 3, limit: int = 5):
    """
    Test the email triage system with a custom date range.
    
    Args:
        days_back: Number of days to look back
        limit: Maximum emails to process
    """
    try:
        test_date = (datetime.now() - timedelta(days=days_back)).strftime('%d-%b-%Y')
        console.print(f"[blue]🔍 Testing with emails from {test_date} (limit: {limit})[/blue]")
        
        # Create secure SSL context
        context = ssl.create_default_context(cafile=certifi.where())
        
        with IMAPClient("imap.gmail.com", ssl=True, ssl_context=context) as imap_client:
            if EMAIL and APP_PASSWORD:
                # Connect and authenticate
                imap_client.login(EMAIL, APP_PASSWORD)
                imap_client.select_folder("INBOX")
                
                # Search for emails from test date
                messages = imap_client.search(["ON", test_date])[:limit]
                response = imap_client.fetch(messages, ["ENVELOPE", "RFC822"])
                
                # Process emails
                emails_test = []
                for msgid, data in response.items():
                    envelope = data[b"ENVELOPE"]
                    subject_raw = envelope.subject.decode(errors="ignore") if envelope.subject else "(No Subject)"
                    subject = decode_subject_value(subject_raw)
                    
                    if envelope.from_ and len(envelope.from_) > 0:
                        from_addr = envelope.from_[0]
                        from_ = f"{from_addr.mailbox.decode()}@{from_addr.host.decode()}"
                    else:
                        from_ = "unknown@unknown.com"
                    
                    raw_bytes = data[b"RFC822"]
                    clean_body = extract_plain_text_from_email(raw_bytes)
                    emails_test.append({"subject": subject, "from": from_, "body": clean_body})
                
                console.print(f"[green]✅ Fetched {len(emails_test)} emails from {test_date}[/green]")
                
                if emails_test:
                    # Analyze with LLM
                    summarizer_test = SummarizerAgent(gravix_client)
                    summaries_test = summarizer_test.summarize_and_prioritize(emails_test)
                    
                    # Generate report
                    reporter_test = ReporterAgent(date_label=f"Test: {test_date}")
                    df_test, md_test = reporter_test.run(summaries_test)
                    
                    # Display results
                    console.rule("[bold cyan]🧪 Test Results: Email Analysis[/bold cyan]")
                    console.print(Markdown(md_test))
                    
                    return df_test
                else:
                    console.print(f"[yellow]No emails found for {test_date}[/yellow]")
                    return pd.DataFrame()
            else:
                console.print("[bold yellow]⚠️  Skipping test: credentials not set[/bold yellow]")
                return pd.DataFrame()
                
    except Exception as e:
        console.print(f"[bold red]❌ Test error:[/bold red] {e}")
        return pd.DataFrame()

# Run the test
test_df = test_custom_date_range(days_back=3, limit=5)

# Display test results
if not test_df.empty:
    console.print(f"\n[cyan]📊 Test DataFrame shape: {test_df.shape}[/cyan]")
    test_df.head()
else:
    console.print("[yellow]No test data to display[/yellow]")

───────────────────────────────────── 🧪 Advanced Testing: Custom Date Range ──────────────────────────────────────

🔍 Testing with emails from 03-Sep-2025 (limit: 5)

✅ Fetched 5 emails from 03-Sep-2025

🧠 Analyzing 5 emails with LLM...

Processing email 1/5...

...

✅ Completed analysis of 5 emails

📈 Email Summary: 0 High, 5 Medium, 0 Low priority

───────────────────────────────────────── 🧪 Test Results: Email Analysis ─────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                      📧 Email Digest for Test: 03-Sep-2025                                      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                           🟡 Medium Priority (5 emails)                                           

           📨 From: deloittesh-jobnotification@noreply44.jobs2web.com  |  Subject: New jobs posted from            
                                           southasiacareers.deloitte.com                                           

Summary: This email notifies you of new job postings at Deloitte, specifically in Bengaluru, India, and invites you
to manage your Job Alerts.                                                                                         

Action Items: Manage your Job Alerts                                                                               

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             📨 From: messages-noreply@linkedin.com  |  Subject: Meera Vanathi commented on your post.             

Summary: Meera Vanathi has commented on one of Rupin Ajay's posts with a positive comment, and the email is        
notifying him about it.                                                                                            

Action Items: View the conversation on LinkedIn                                                                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             📨 From: newsletter@opencv.org  |  Subject: OpenCV Newsletter: Updates and Opportunities              

Summary: The OpenCV newsletter provides updates on projects using OpenCV, opportunities to get involved with the   
community, and resources for learning and contributing.                                                            

Action Items: Shop Official OpenCV Merchandise, Join the OpenCV Community Slack Server, Follow OpenCV on various   
social media platforms                                                                                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                    📨 From: no-reply@duolingo.com  |  Subject: You’re invited to Duocon 2025!                     

Summary: Duolingo is inviting users to attend a free virtual conference, Duocon 2025, on September 16th, featuring 
updates and announcements about new courses, features, and assessments.                                            

Action Items: Register for the event by checking out the official website                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
            📨 From: team@datacamp.com  |  Subject: New Course: GDPR in Practice: Compliance and Fines             

Summary: DataCamp is announcing a new course on GDPR in Practice: Compliance and Fines, as part of their Data & AI 
Literacy Month. The email also promotes other courses, webinars, and articles related to data science and AI.      

Action Items: Learn about the new course, explore other resources, consider upskilling your organization           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

📊 Test DataFrame shape: (5, 6)

## Conclusion

### What We've Accomplished

In this comprehensive recipe, we've built a complete email triage ecosystem:

1. **Secure Email Processing** - Secure IMAP integration with SSL/TLS encryption
2. **Intelligent Agent Architecture** - Three specialized agents working in orchestration
3. **AI-Powered Analysis** - Real-world email understanding with Gravix Layer LLM
4. **Multi-Format Reporting** - Structured outputs for both humans and systems

### The Power of Agentic Email Processing

This agentic approach represents a fundamental shift in email management:

- **Automation**: Eliminates manual email sorting and prioritization
- **Intelligence**: Context-aware analysis that understands email content
- **Scalability**: Handles high-volume processing with configurable limits
- **Flexibility**: Customizable priority criteria and output formats
- **Integration**: Multiple export options for existing business systems

### Next Steps

To continue your email automation journey:

1. **Customize Priority Logic** - Adjust classification criteria for your specific needs
2. **Scale Your Deployment** - Add database integration and multi-account support
3. **Enhance Features** - Implement calendar integration and advanced analytics
4. **Deploy in Production** - Add monitoring, logging, and enterprise security

### Key Takeaways

- **Agentic architecture simplifies complex workflows** - Each agent has a focused responsibility
- **LLM integration enables intelligent automation** - AI that truly understands email content
- **Multiple output formats ensure flexibility** - Works with any downstream system
